In [7]:
import os, time
import cv2
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
import random
from pathlib import Path

#time_str = (datetime.datetime.now()).strftime("%Y%m%d")
time_str = '20190128'
xml_dir = '/data/darknet/python/' + time_str + '_xml_fillter_obj/'
SEED_XML_DIR = xml_dir
GENE_XML_DIR = SEED_XML_DIR


# RT:RightTop 
# LB:LeftBottom 
# bbox: [xmin, xax, ymin, ymax]
def IOU(bbox_a, bbox_b):
    '''
    W = min(A.RT.x, B.RT.x) - max(A.LB.x, B.LB.x) 
    H = min(A.RT.y, B.RT.y) - max(A.LB.y, B.LB.y) 
    if W <= 0 or H <= 0: 
        return 0 
    SA = (A.RT.x - A.LB.x) * (A.RT.y - A.LB.y) 
    SB = (B.RT.x - B.LB.x) * (B.RT.y - B.LB.y) 
    cross = W * H return cross/(SA + SB - cross)
    '''
    W = min(bbox_a[1], bbox_b[1]) - max(bbox_a[0], bbox_b[0]) 
    H = min(bbox_a[3], bbox_b[3]) - max(bbox_a[2], bbox_b[2]) 
    if W <= 0 or H <= 0: 
        return 0
    SA = (bbox_a[1] - bbox_a[0]) * (bbox_a[3] - bbox_a[2]) 
    SB = (bbox_b[1] - bbox_b[0]) * (bbox_b[3] - bbox_b[2])  
    cross = W * H 
    return cross/(SA + SB - cross)

def get_obj_from_xml(xml):
    in_file = open(xml_name)
    tree=ET.parse(in_file)
    root = tree.getroot()
    return [obj for obj in root.iter('object')]

def get_obj_from_image_file(file, bbox):
    img = cv2.imread(file)
    img_obj = img[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])]
    return img_obj

def get_bboxes_from_etree(etree):
    root = etree.getroot()  
    objects = root.findall('object')
    bboxes = []
    for obj in objects:
        xmlbox = obj.find('bndbox')
        b = [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
        bboxes.append(b)
    return bboxes

seed_xml_names = os.listdir(SEED_XML_DIR)
seed_xml_names.sort()

classes_ = ['bus', 'car', 'truck', 'motorbike', 'bicycle', 'person']
#classes_ = ['person']

for xml_name in seed_xml_names:    
    if(not xml_name.endswith('.xml')):
        continue
    if(not (int(xml_name[-15:-4]) >= 461000)):
        continue   
    #img_data = cv2.imread(SEED_IMG_DIR + xml_name[:-3] + 'jpg')
    print(SEED_XML_DIR + xml_name)
    in_file = open(SEED_XML_DIR + xml_name)
    tree=ET.parse(in_file)

    root = tree.getroot()
    size = root.find('size')   
    boxes = []
    b = []  
    wrong_detection_objs = [[599, 639, 139, 196], [104, 122, 80, 98]]

    array_object = root.findall('object')
    for obj in array_object:
        erro_obj_removed = False
        difficult = obj.find('difficult').text
        cls_ = obj.find('name').text
        if cls_ not in classes_ or int(difficult)==1:
            continue
        xmlbox = obj.find('bndbox')
        b = [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
        for erro_bbox in wrong_detection_objs:
            if(IOU(erro_bbox, b) > 0.4):
                print(IOU(erro_bbox, b))
                print('removed')
                root.remove(obj)
                erro_obj_removed = True
                break
        if(erro_obj_removed):
            continue
        '''
        #filter size which smaller than 16X16 fit for SSD300
        bbox_width = b[1] - b[0]
        bbox_heigt = b[3] - b[2]
        if(not(bbox_width > 16 or bbox_heigt > 16)):
            print('remove:{0}:{1}'.format(bbox_width, bbox_heigt))
            root.remove(obj)
            continue 
        else:
            print('add:{0}:{1}'.format(bbox_width, bbox_heigt))
        '''    
    tree.write(GENE_XML_DIR + xml_name)

/data/darknet/python/20190128_xml_fillter_obj/20190123161747.003630_00000461000.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123161747.044919_00000461001.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123161747.058555_00000461002.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123161747.071982_00000461003.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123161747.085121_00000461004.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123161747.098213_00000461005.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123161747.111791_00000461006.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123161747.126126_00000461007.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123161747.139399_00000461008.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123161747.153495_00000461009.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123161747.167635_00000461010.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123161747.182465_00000461

/data/darknet/python/20190128_xml_fillter_obj/20190123161751.959958_00000461353.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123161751.974624_00000461354.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123161751.988694_00000461355.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123161752.002630_00000461356.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123161752.016818_00000461357.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123161752.031450_00000461358.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123161752.045777_00000461359.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123161752.059048_00000461360.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123161752.072771_00000461361.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123161752.087369_00000461362.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123161752.100674_00000461363.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123161752.114336_00000461

/data/darknet/python/20190128_xml_fillter_obj/20190123161756.978576_00000461713.xml
0.56738768718802
removed
/data/darknet/python/20190128_xml_fillter_obj/20190123161756.992346_00000461714.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123161757.006273_00000461715.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123161757.019577_00000461716.xml
0.6502940159114493
removed
/data/darknet/python/20190128_xml_fillter_obj/20190123161757.032471_00000461717.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123161757.046504_00000461718.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123161757.060715_00000461719.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123161757.074570_00000461720.xml
0.8416389811738649
removed
/data/darknet/python/20190128_xml_fillter_obj/20190123161757.088535_00000461721.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123161757.102130_00000461722.xml
0.8616780045351474
removed
/data/darknet/python/20190128_xml_fillter_obj/20190123

/data/darknet/python/20190128_xml_fillter_obj/20190123163934.044704_00000463104.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123163934.057610_00000463105.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123163934.070664_00000463106.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123163934.083231_00000463107.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123163934.095986_00000463108.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123163934.108813_00000463109.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123163934.121870_00000463110.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123163934.136868_00000463111.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123163934.256455_00000463112.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123163934.274498_00000463113.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123163934.287592_00000463114.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123163934.300552_00000463

/data/darknet/python/20190128_xml_fillter_obj/20190123163941.690786_00000463543.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123163941.720181_00000463544.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123163941.734079_00000463545.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123163941.748105_00000463546.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123163941.761174_00000463547.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123163941.775594_00000463548.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123163941.788885_00000463549.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123163941.803347_00000463550.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123163941.818071_00000463551.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123163941.832578_00000463552.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123163941.846221_00000463553.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123163941.859844_00000463

/data/darknet/python/20190128_xml_fillter_obj/20190123163950.150306_00000464042.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123163950.168644_00000464043.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123163950.182506_00000464044.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123163950.237650_00000464045.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123163950.251151_00000464046.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123163950.265962_00000464047.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123163950.279820_00000464048.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123163950.294112_00000464049.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123163950.307762_00000464050.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123163950.321570_00000464051.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123163950.334966_00000464052.xml
/data/darknet/python/20190128_xml_fillter_obj/20190123163950.348206_00000464